# Job Glue ETL Adquirencia - Fraude

Fuentes: 
- Datalake prod: uala-arg-datalake-analytics-prod

In [15]:
!pip install -q awswrangler

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [16]:
import awswrangler as wr
import pandas as pd
import boto3
import time

In [17]:
s3 = boto3.resource('s3')
glue = boto3.client('glue')

In [19]:
%%writefile job_adquirencia.py

#####################
####### Libs ########
#####################
#!pip install -upgrade boto3
print('Importar librerías')
import numpy as np
import pandas as pd
import json
import ast
#import math
import gc
import os
import sys
from itertools import chain
import awswrangler as wr

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta, SU

from awsglue.dynamicframe import DynamicFrame
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from awsglue.job import Job
from awsglue.context import GlueContext

from pyspark.sql.types import *
from pyspark.context import SparkContext
from pyspark.conf import SparkConf
import pyspark.sql.functions as F

import boto3
ssm = boto3.client('ssm') 
###
os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages=com.amazonaws:aws-java-sdk-bundle:1.11.271,org.apache.hadoop:hadoop-aws:3.1.2 pyspark-shell"
###


#####################
####### Params ######
#####################
print('Leer parámetros')
args = getResolvedOptions(sys.argv,
                          ['adquirencia_bucket',
                           'adquirencia_key_inf', 
                           'datalake_prod_bucket', 
                           'transactions_key',    # bucket prod datalake
                           'gp1010_key',          # bucket prod datalake
                           'cotizaciones_bucket', # momentáneamente en auxiliar
                           'cotizaciones_key',
                           'accountid_gp_bucket', # momentáneamente en auxiliar
                           'accountid_gp_key'
                           #'last_run_ps_name',
                           #'kms_key_arn'
                           ])

adquirencia_bucket   = args['adquirencia_bucket']
adquirencia_key_inf  = args['adquirencia_key_inf']
datalake_prod_bucket = args['datalake_prod_bucket']
transactions_key     = args['transactions_key']
gp1010_key           = args['gp1010_key']
cotizaciones_bucket  = args['cotizaciones_bucket']
cotizaciones_key     = args['cotizaciones_key']
accountid_gp_bucket  = args['accountid_gp_bucket']
accountid_gp_key     = args['accountid_gp_key']

#kms_key_arn         = args['kms_key_arn']
#last_run_ps_name = args['last_run_ps_name']

now = datetime.now().date()
print(f'Now = {now}')
now = pd.to_datetime(now)

prior_6_month = (now - pd.DateOffset(months=6)).strftime('%Y-%m-%d')
print(f'prior 6m = {prior_6_month}')

one_week_prior = now - pd.Timedelta(7, 'd')
#one_week_prior = one_week_prior.strftime('%Y-%m-%d')
print(f'prior 1 week: {one_week_prior}')


######################
##### Conexiones #####
######################

######### Boto  #############
print('Request keys boto3 client & resource')
#glue = boto3.client('glue')
sts = boto3.client('sts')

# Conexiones para listar archivos 
response = sts.assume_role(
    RoleArn         = 'arn:aws:iam::514405401387:role/aws-rol-ml-read-stage-prod', #es el rol que existe en produccion por el cual "nos hacemos pasar" para acceder a los buckets de s3
    RoleSessionName = 'sesion-adq', # nombre que le damos a la sesión
    DurationSeconds = 3600 # es el tiempo que dura la sesion por default si no especificamos este parámetro.
)

s3c = boto3.client('s3',
    aws_access_key_id     = response['Credentials']['AccessKeyId'],
    aws_secret_access_key = response['Credentials']['SecretAccessKey'],
    aws_session_token     = response['Credentials']['SessionToken']
)

s3r = boto3.resource('s3',
    aws_access_key_id     = response['Credentials']['AccessKeyId'],
    aws_secret_access_key = response['Credentials']['SecretAccessKey'],
    aws_session_token     = response['Credentials']['SessionToken']
)


######### Spark  #############

print('Request keys Spark y configuracion')
response_spark = sts.assume_role(RoleArn = 'arn:aws:iam::514405401387:role/aws-rol-ml-read-stage-prod', #es el rol que existe en produccion por el cual "nos hacemos pasar" para acceder a los buckets de s3
                                 RoleSessionName = 'sesion-adq-spark', # nombre que le damos a la sesión
                                 DurationSeconds = 3600 # es el tiempo que dura la sesion por default si no especificamos este parámetro.
                                )

spark_conf = SparkConf().setAll([
  ("spark.hadoop.fs.s3.enableServerSideEncryption", "true")
  #("spark.hadoop.fs.s3.serverSideEncryption.kms.keyId", kms_key_id)
                                ])

sc = SparkContext(conf=spark_conf)
glueContext = GlueContext(sc)
spark = glueContext.spark_session
logger = glueContext.get_logger()

spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider", 
                                     "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key",     response_spark["Credentials"]["AccessKeyId"])
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key",     response_spark["Credentials"]["SecretAccessKey"])
spark._jsc.hadoopConfiguration().set("fs.s3a.session.token",  response_spark["Credentials"]["SessionToken"])


#####################
##### Functions #####
#####################

def list_objects_function(buckets_, keys_, retrieve_date='', date_logic='equal'):
    """
    date_ (str) : Fecha para buscar parquets.
    date_logic (str) :  Si 'last', devuelve una lista con los path de los parquets de la carpeta más reciente y la fecha correspondiente. 
                        Si 'great_equal', devuelve una lista con los path de los parquets de las carpetas >= date_.
                        Si 'equal',  devuelve una lista con los path de los parquets de las carpetas == date_.
                        Si 'all', devuelve una lista con todos los paths de los parquets 
    """
   

    paginator = s3c.get_paginator('list_objects_v2')
    pages     = paginator.paginate(Bucket=buckets_, 
                                   Prefix=keys_)
    files_in_bucket=[]
    for page in pages:
        files_page = [key['Key'] for key in page['Contents']]
        files_in_bucket+=files_page
    #s3a al leer cross account, sino  s3
    files_objets = [f"s3a://{buckets_}/" + i for i in files_in_bucket if i.find('.parquet') >= 0]
    
    df_bucket_files = pd.DataFrame({
        'key': [i[:(i.find('dt=') + 14)] for i in files_objets],
        'path': files_objets,
        'date': pd.to_datetime([i[(i.find('dt=') + 3):(i.find('dt=') + 13)] for i in files_objets])
    })

    if date_logic == 'last':
        max_date = df_bucket_files['date'].max().strftime('%Y-%m-%d')
        files = df_bucket_files.query('date == @max_date')['path'].tolist()
        return files, max_date
    # Para transactions:
    elif date_logic == 'great_equal':
        files = df_bucket_files.query(
            'date >= @retrieve_date')['path'].to_list()
        return files
    
    elif date_logic == 'equal':
        files = df_bucket_files.query(
            'date == @retrieve_date')['path'].to_list()
        return files
    
    elif date_logic == 'all':
        return df_bucket_files['path'].to_list()

    
    

################################
##  accountID <--> accountGP  ##
################################
print('Lectura mapeo accountID - accountGP')

mapeo_id_gp = wr.s3.read_parquet(f's3://{accountid_gp_bucket}/{accountid_gp_key}/account_id_gp.parquet')
df_mapeo_id_gp = spark.createDataFrame(mapeo_id_gp)
df_mapeo_id_gp = df_mapeo_id_gp.withColumn('externalid', df_mapeo_id_gp.externalid.cast('string'))


################################
#######  Lectura users   #######
################################

print('Lectura users')

path_users = list_objects_function(buckets_ = datalake_prod_bucket, 
                                   keys_ = gp1010_key, 
                                   retrieve_date = '', 
                                   date_logic = 'all')

columnas_usr = ['numero_cuenta', 'estado_cuenta', 'fecha_alta', 'fecha_nacimiento', 'sexo', 'provincia']
df_usr = spark.read.parquet(*path_users).select(columnas_usr)

print(f'Hay {df_usr.count()} users')
df_usr = df_usr.dropDuplicates(['numero_cuenta'])
print(f'Hay {df_usr.count()} users luego de sacar duplicados')

# dtypes
df_usr = (df_usr
          .withColumn('numero_cuenta', df_usr.numero_cuenta.cast('string'))
          .withColumn('fecha_alta', F.to_date(df_usr.fecha_alta,"yyyy-MM-dd"))
          .withColumn('fecha_nacimiento', F.to_date(df_usr.fecha_nacimiento,"yyyy-MM-dd"))
          .withColumn('provincia', df_usr.provincia.cast('int'))
         )

# Edad y antiguedad
df_usr = (df_usr
          .withColumn('edad', F.floor(F.datediff(F.current_date(), F.col('fecha_nacimiento'))/365.25))
          .withColumn('antiguedad', F.datediff(F.current_date(), F.col('fecha_alta')))
         )

# Provincia int a str
prov_dict = {
    1: 'CAPITAL_FEDERAL',
    2: 'BUENOS_AIRES',  # GRAN BUENOS AIRES
    3: 'BUENOS_AIRES',
    4: 'OTROS',  # CATAMARCA
    5: 'CORDOBA',
    6: 'NORTE_ESTE',  # CORRIENTES
    7: 'NORTE_ESTE',  # CHACO
    8: 'OTROS',  # CHUBUT
    9: 'OTROS',  # ENTRE RIOS
    10: 'NORTE_ESTE',  # FORMOSA
    11: 'NORTE',  # JUJUY
    12: 'OTROS',  # LA PAMPA
    13: 'OTROS',  # LA RIOJA
    14: 'MENDOZA',
    15: 'NORTE_ESTE',  # MISIONES
    16: 'OTROS',  # NEUQUEN
    17: 'OTROS',  # RIO NEGRO
    18: 'NORTE',  # SALTA
    19: 'OTROS',  # SAN JUAN
    20: 'OTROS',  # SAN LUIS
    21: 'SANTA_FE',
    22: 'OTROS',  # SANTA CRUZ
    23: 'OTROS',  # SANTIAGO DEL ESTERO
    24: 'OTROS',  # TIERRA DEL FUEGO
    25: 'NORTE'  # TUCUMAN
}

prov_mapping = F.create_map([F.lit(x) for x in chain(*prov_dict.items())])
df_usr = df_usr.withColumn('provincia', prov_mapping[df_usr['provincia']])


################################
##### Lectura cotizaciones #####
################################

print('Lectura cotizaciones')

cotizaciones = wr.s3.read_parquet(f's3://{cotizaciones_bucket}/{cotizaciones_key}/df_cotizacion.parquet')

print(f'Fecha más nueva con cotizaciones: {cotizaciones.fecha.max()}')
df_cotizaciones = spark.createDataFrame(cotizaciones)
df_cotizaciones = df_cotizaciones.withColumn('fecha', F.to_date(df_cotizaciones.fecha))


################################
##### Lectura transactions #####
################################

print('Lectura transactions')
path_transactions = list_objects_function(buckets_ = datalake_prod_bucket, 
                                          keys_ = transactions_key, 
                                          retrieve_date = prior_6_month, 
                                          date_logic = 'great_equal')
print(f'{len(path_transactions)} parquets de trx encontrados')

columnas_transactions = ["transaction_date","account_from", "account_to", "amount", 
                         "transaction_type", "status", "transaction_id"]

df_trx = spark.read.parquet(*path_transactions).select(columnas_transactions)

df_trx = df_trx.dropDuplicates(['transaction_id'])
print(f'Hay {df_trx.count()} transacciones luego de sacar duplicados')


### Transformaciones ###
########################

#df = df.withColumn('transaction_date', F.to_timestamp(df.transaction_date, "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"))
df_trx = df_trx.withColumn('transaction_date', F.to_date(df_trx.transaction_date))
# Filtramos por status. Ver qué hacer con "CANCELED"
df_trx = df_trx.filter(F.col('status').isin(['AUTHORIZED', 'REJECTED']))

### Generar columna mes agrupacion
def gen_interval(date):
    cotas = [(now - pd.DateOffset(months=i)) for i in range(1,7)]
    if (date >= cotas[0]):
        return '6'
    elif date >= cotas[1]:
        return '5'
    elif date >= cotas[2]:
        return '4'
    elif date >= cotas[3]:
        return '3'
    elif date >= cotas[4]:
        return '2'
    elif date >= cotas[5]:
        return '1'

col_date_interval = F.udf(gen_interval, StringType())
df_trx = df_trx.withColumn('transaction_month', col_date_interval('transaction_date'))
print(df_trx.select(['transaction_date', 'transaction_month']).show(2))

### Pasar a USD
df_trx = df_trx.join(df_cotizaciones,  df_trx.transaction_date == df_cotizaciones.fecha, how='left')
df_trx = (df_trx
          .drop(F.col('fecha'))
          .withColumn('amount_usd', F.col('amount')/F.col('venta_uala'))
         )


# USER_TO_USER_IN
df_trx_user_user_in = df_trx.filter(df_trx.transaction_type == "USER_TO_USER")
df_trx_user_user_in = df_trx_user_user_in.drop(F.col('account_from'))
df_trx_user_user_in = df_trx_user_user_in.withColumnRenamed('account_to','account_id')
df_trx_user_user_in = df_trx_user_user_in.withColumn('transaction_type', F.lit('USER_TO_USER_IN'))
df_trx = df_trx.drop(F.col('account_to'))
df_trx = df_trx.withColumnRenamed('account_from','account_id')
df_trx = df_trx.union(df_trx_user_user_in)


### 6 meses ###

print('Creación variables 6 meses')
df_trx_6m = (df_trx
             # a string porque sino groupby lo agrega
             .withColumn('cashin_tdc_6m', F.when((F.col('transaction_type') == 'CASH_IN_TDC') & (F.col('status') == 'AUTHORIZED'), F.col('amount_usd')).otherwise(0))
             .withColumn('cashin_to_user_6m', F.when((F.col('transaction_type') == 'USER_TO_USER_IN') & (F.col('status') == 'AUTHORIZED'), F.col('amount_usd')).otherwise(0))
             .withColumn('cashin_cvu_6m', F.when((F.col('transaction_type') == 'CASH_IN_CVU') & (F.col('status') == 'AUTHORIZED'), F.col('amount_usd')).otherwise(0))
             .withColumn('cashin_investments_withdraw_6m', F.when((F.col('transaction_type') == 'INVESTMENTS_WITHDRAW') & (F.col('status') == 'AUTHORIZED'), F.col('amount_usd')).otherwise(0))
             .withColumn('cashout_purchase_6m', F.when((F.col('transaction_type') == 'CONSUMPTION_POS') & (F.col('status') == 'AUTHORIZED'), F.col('amount_usd')).otherwise(0))
             .withColumn('cashout_cvu_6m', F.when((F.col('transaction_type') == 'CASH_OUT_CVU') & (F.col('status') == 'AUTHORIZED'), F.col('amount_usd')).otherwise(0))
             .withColumn('cashout_user_to_user_6m', F.when((F.col('transaction_type') == 'USER_TO_USER') & (F.col('status') == 'AUTHORIZED'), F.col('amount_usd')).otherwise(0))
             .withColumn('cashout_telerecarga_6m', F.when((F.col('transaction_type') == 'TELERECARGAS_CARGA') & (F.col('status') == 'AUTHORIZED'), F.col('amount_usd')).otherwise(0))
             .withColumn('cashout_automatic_debit_6m', F.when((F.col('transaction_type') == 'AUTOMATIC_DEBIT') & (F.col('status') == 'AUTHORIZED'), F.col('amount_usd')).otherwise(0))
             .withColumn('cashout_withdraw_atm_6m', F.when((F.col('transaction_type') == 'WITHDRAW_ATM') & (F.col('status') == 'AUTHORIZED'), F.col('amount_usd')).otherwise(0))
             .withColumn('cashout_rechazadas_con_ad_6m', F.when((F.col('transaction_type') == 'AUTOMATIC_DEBIT') & (F.col('status') == 'REJECTED'), F.col('amount_usd')).otherwise(0))
             .withColumn('cashout_rechazadas_sin_ad_6m', F.when((F.col('transaction_type').isin(['CONSUMPTION_POS', 'CASH_OUT_CVU', 'USER_TO_USER', 'TELERECARGAS_CARGA', 'WITHDRAW_ATM'])) &
                                                                (F.col('status') == 'REJECTED'), F.col('amount_usd')).otherwise(0))
             )

print('Groupby por accountID y mes')
cols_cash = ['account_id', 'transaction_month'] + [s for s in df_trx_6m.columns if 'cash' in s]
df_trx_6m = (df_trx_6m.select(cols_cash).groupBy(['account_id', 'transaction_month']).sum()).toDF(*cols_cash)
print('primer groupby 6m exitoso')
#print(f'{df_trx_6m.count()} users con trx 6m')

df_pandas = df_trx_6m.toPandas()
wr.s3.to_parquet(
                 df=df_pandas,
                 path=(f's3://{adquirencia_bucket}/data/test/df_6m.parquet')
                )



# ##
# print('Copiamos solo 1 mes de trx')
# cols_cash = ['account_id', 'transaction_month'] + [s.replace('6','1') for s in df_trx_6m.columns if 'cash' in s]
# df_trx_1m = df_trx_6m.filter(df_trx_6m.transaction_month == '6').toDF(*cols_cash)
# #print(f'{df_trx_1m.count()} users con trx 1m')

# ##

# print('Groupby por accountID')
# cols_cash = ['account_id'] + [s for s in df_trx_6m.columns if 'cash' in s]

# # df_trx_6m = df_trx_6m.select(cols_cash).groupBy('account_id').agg(
# #     F.expr('percentile(cashin_tdc_6m, array(0.5))')[0].alias('cashin_tdc_6m'),
# #     F.expr('percentile(cashin_to_user_6m, array(0.5))')[0].alias('cashin_to_user_6m'),
# #     F.expr('percentile(cashin_cvu_6m, array(0.5))')[0].alias('cashin_cvu_6m'),
# #     F.expr('percentile(cashin_investments_withdraw_6m, array(0.5))')[0].alias('cashin_investments_withdraw_6m'),
# #     F.expr('percentile(cashout_purchase_6m, array(0.5))')[0].alias('cashout_purchase_6m'),
# #     F.expr('percentile(cashout_cvu_6m, array(0.5))')[0].alias('cashout_cvu_6m'),
# #     F.expr('percentile(cashout_user_to_user_6m, array(0.5))')[0].alias('cashout_user_to_user_6m'),
# #     F.expr('percentile(cashout_telerecarga_6m, array(0.5))')[0].alias('cashout_telerecarga_6m'),
# #     F.expr('percentile(cashout_automatic_debit_6m, array(0.5))')[0].alias('cashout_automatic_debit_6m'),
# #     F.expr('percentile(cashout_withdraw_atm_6m, array(0.5))')[0].alias('cashout_withdraw_atm_6m'),
# #     F.expr('percentile(cashout_rechazadas_con_ad_6m, array(0.5))')[0].alias('cashout_rechazadas_con_ad_6m'),
# #     F.expr('percentile(cashout_rechazadas_sin_ad_6m, array(0.5))')[0].alias('cashout_rechazadas_sin_ad_6m')
# #     )

# # Frequency
# df_freq = df_trx.select(['account_id', 'transaction_date']).filter((F.col('transaction_date') >= one_week_prior))
# print(f'trx última semana: {df_freq.count()}')
# df_freq = df_freq.groupby(['account_id', 'transaction_date']).count()
# print(df_freq.show(2))
# df_freq = df_freq.groupby(['account_id']).count().toDF(['account_id', 'f_1week'])
# print(df_freq.show(2))



# #####################
# ### Merge todo ######
# #####################
# print('Mergear todos los df')
# df_inferencias = (df_usr
#                   .join(df_trx_6m, on='account_id', how='left')
#                   .join(a, on='accountgp', how='left', suffixes=('_6m', '_1m'))
#                   .merge(df_cashout_6m, on='accountgp', how='left')
#                   .merge(df_cashout_1m, on='accountgp', how='left', suffixes=('_6m', '_1m'))
#                   .merge(df_churn, on='accountgp', how='left')
#                   )

# df_inferencias.fillna(0, inplace=True)


# #####################
# ##### Dummies #######
# #####################

# print('Generar dummies para "sexo" y "provincia"')
# df_inferencias["sexo_"] = df_inferencias["sexo"] #mantener original
# df_inferencias["sexo_"] = df_inferencias["sexo_"].astype('category')
# df_inferencias["provincia_"] = df_inferencias["provincia"] #mantener original
# df_inferencias["provincia_"] = df_inferencias["provincia_"].astype('category')
# df_inferencias = pd.get_dummies(df_inferencias, columns=['sexo_', 'provincia_'], prefix_sep='', drop_first=True)

# #####################
# #  Inferencia coef  #
# #####################
# print('Inferencia')

# coeficientes = ssm.get_parameter(Name='/test_adquirencia_coeficientes')['Parameter']['Value']
# #coef = ast.literal_eval(str_)
# coeficientes = json.loads(coeficientes)

# coef_s = pd.Series(coeficientes).astype(float) # Vienen como str del parameter store
# cols_coef = coef_s.drop('b0').index.tolist() # lista con las columnas que llevan coeficiente

# df_inferencias['pred'] = (df_inferencias[cols_coef] # Nos quedamos solo con las columnas con coeficientes
#                            .dot(coef_s[cols_coef]) # Multiplicación matricial con los coeficientes
#                            .add(coef_s['b0']) # Sumamos b0 que no forma parte de la lista cols_coefi
#                           )

# #####################
# ##### Save s3 #######
# #####################
# print('Guardar df final en s3')
# wr.s3.to_parquet(
#      df=df_inferencias,
#      path=(f's3://{adquirencia_bucket}/{adquirencia_key_inf}/dt={now.strftime("%Y-%m-%d")}/df_inference_{now.strftime("%Y-%m-%d")}.parquet')
#      #partition_efls=['col2']
# )


# ######################
# # Actualizar LAST_RUN #
# ######################
# print('Actualizar last run')
# ssm.put_parameter(
#          Name=last_run_ps_name,
#          Value=now.strftime("%Y-%m-%d"),
#          Type='String',
#          Overwrite=True)
# print('Last run actualizado')

Overwriting job_adquirencia.py


# Subir archivos

In [20]:
adquirencia_bucket   = 'test-datascience-adquirencia-fraude'
# Guardar el archivo .py
s3.meta.client.upload_file('job_adquirencia.py', 
                            adquirencia_bucket, #bucket
                           'artifacts/code/job_adquirencia.py')
print('.py uploaded')

.py uploaded


### Archivos auxiliares: 
Por ahora los subimos manualmente:

# Job Glue

Parámetros: 

In [21]:
job_name = 'job_adquirencia_fraude'

In [22]:
adquirencia_bucket   = 'test-datascience-adquirencia-fraude'
adquirencia_key_inf  = 'inferences'
datalake_prod_bucket = 'uala-arg-datalake-analytics-prod'
transactions_key     = 'ar/tb_ar_core_transactions'
gp1010_key           = 'ar/tb_ar_gp_t1010'
cotizaciones_bucket  = 'test-datascience-adquirencia-fraude'
cotizaciones_key     = 'data/auxiliar'
accountid_gp_bucket  = 'test-datascience-adquirencia-fraude'
accountid_gp_key     = 'data/auxiliar'

In [23]:
# borrar job
glue.delete_job(
    JobName=job_name
)

{'JobName': 'job_adquirencia_fraude',
 'ResponseMetadata': {'RequestId': 'aff1ca43-d750-4855-8dfa-2701579da877',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 16 Jul 2021 14:02:17 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '36',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'aff1ca43-d750-4855-8dfa-2701579da877'},
  'RetryAttempts': 0}}

In [24]:
job = glue.create_job(Name=job_name, 
                      GlueVersion='2.0',
                      Role='iam_r_uala_arg_datalake_stage_glue',
                      Command={'Name': 'glueetl',
                               'ScriptLocation': f's3://{adquirencia_bucket}/artifacts/code/job_adquirencia.py'},
                      DefaultArguments={
                        '--additional-python-modules': 'pip,setuptools,pyarrow==2,awswrangler==2.8.0,numpy==1.19.1'},
                      MaxCapacity=1
                      )

AccessDeniedException: An error occurred (AccessDeniedException) when calling the CreateJob operation: User: arn:aws:sts::465900123368:assumed-role/iam_r_uala_arg_datalake_stage_sagemaker/SageMaker is not authorized to perform: iam:PassRole on resource: arn:aws:iam::465900123368:role/iam_r_uala_arg_datalake_stage_glue

In [111]:
job_run = glue.start_job_run(
    JobName = job_name,
    Arguments = {
         '--adquirencia_bucket':   adquirencia_bucket,
         '--adquirencia_key_inf':  adquirencia_key_inf,
         '--datalake_prod_bucket': datalake_prod_bucket,
         '--transactions_key':     transactions_key,
         '--gp1010_key':           gp1010_key,
         '--cotizaciones_bucket':  cotizaciones_bucket,
         '--cotizaciones_key':     cotizaciones_key,
         '--accountid_gp_bucket':  accountid_gp_bucket,
         '--accountid_gp_key':     accountid_gp_key
    } 
)

In [112]:
#print(job_run)

In [113]:
from IPython.display import clear_output

MAX_WAIT_TIME=time.time() + 60*10 # 1 hour
max_time = time.time() + MAX_WAIT_TIME
while time.time() < max_time:
    response=glue.get_job_run(JobName=job_name, RunId=job_run['JobRunId'])
    status = response['JobRun']['JobRunState']
    clear_output(wait=True)
    print('Job run: {}'.format(status))
    
    if status == 'SUCCEEDED' or status == 'FAILED':
        break
        
    time.sleep(5)

Job run: SUCCEEDED


In [7]:
test = pd.read_parquet(f's3://{adquirencia_bucket}/data/test/df_6m.parquet')


In [8]:
test.sample(5)

,account_id,transaction_month,cashin_tdc_6m,cashin_to_user_6m,cashin_cvu_6m,cashin_investments_withdraw_6m,cashout_purchase_6m,cashout_cvu_6m,cashout_user_to_user_6m,cashout_telerecarga_6m,cashout_automatic_debit_6m,cashout_withdraw_atm_6m,cashout_rechazadas_con_ad_6m,cashout_rechazadas_sin_ad_6m
5364018,d3aee692-5fe2-43a5-a915-0c8890acd648,6,0.0,0.0,0.000000,0.0,6.384499,0.0,0.467028,0.0,0.597422,0.0,0.0,0.000000
2013302,755ec547-9792-478e-83b5-6ba2424f2fe2,6,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
5911028,469d0097-22e0-49c5-a42a-4357b009ac05,6,0.0,0.0,106.482346,0.0,103.410611,0.0,0.000000,0.0,2.758733,0.0,0.0,0.000000
4617969,b396d35f-8deb-4369-9227-484bf998b8fe,5,0.0,0.0,0.000000,0.0,0.217925,0.0,0.000000,0.0,0.000000,0.0,0.0,25.427291
4745293,7e30becc-9a53-42a9-acfb-f78765d9dfcd,1,0.0,0.0,0.000000,0.0,3.969481,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000


In [9]:
test.shape

(7082407, 14)

In [10]:
test.transaction_month.value_counts()

6    1310577
5    1301640
4    1237987
3    1124178
2    1059513
1    1048512
Name: transaction_month, dtype: int64

In [12]:
%%time
test.drop('transaction_month',axis=1).groupby('account_id',as_index=False).median()

CPU times: user 11.7 s, sys: 810 ms, total: 12.5 s
Wall time: 12.6 s


,account_id,cashin_tdc_6m,cashin_to_user_6m,cashin_cvu_6m,cashin_investments_withdraw_6m,cashout_purchase_6m,cashout_cvu_6m,cashout_user_to_user_6m,cashout_telerecarga_6m,cashout_automatic_debit_6m,cashout_withdraw_atm_6m,cashout_rechazadas_con_ad_6m,cashout_rechazadas_sin_ad_6m
0,000005b1-6bd9-473e-aecf-3c23739ef825,0.000000,0.000000,10.781749,0.000000,9.123516,0.000000,0.000000,0.0,0.0,0.0,0.0,18.370456
1,00000750-eb3a-4dc1-a91e-701ac44aeec1,0.000000,24.364572,293.501876,268.083426,0.000000,268.083426,0.000000,0.0,0.0,0.0,0.0,0.000000
2,00000f06-ce69-4024-96af-3d44f9fc7853,4.670278,0.093406,0.186811,0.000000,0.000000,0.000000,0.186811,0.0,0.0,0.0,0.0,0.000000
3,00002479-2ed2-4c6d-9cc1-2e827857bf3d,3.026940,0.000000,0.000000,0.000000,0.000000,1.025431,0.000000,0.0,0.0,0.0,0.0,0.000000
4,000024d1-43e8-4e26-b01f-22aa7759e678,2.394407,6.085457,0.000000,0.000000,2.582019,0.000000,1.648693,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884166,ffffe851-156f-4029-b3e4-6dc09fecffec,126.768868,100.897992,0.000000,0.000000,1.670597,177.672956,0.000000,0.0,0.0,0.0,0.0,0.000000
1884167,ffffeb20-8b5b-44e9-bdb1-e124ee152a3a,51.939845,0.000000,9.340557,0.000000,47.457651,13.590510,0.000000,0.0,0.0,0.0,0.0,4.670278
1884168,ffffebcd-a142-4cc6-a922-c444dc5b1f8e,0.000000,11.262996,0.000000,0.000000,0.000000,1.505420,0.000000,0.0,0.0,0.0,0.0,0.000000
1884169,ffffefc0-1686-4997-9a36-f8ec6a0f3e1b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.202881
